In [7]:
# prompt: We got 2 files correct_keypoints_normalized.npy and wrong one. Reshape the data to 3d arrays (samples, 30,132) and labels it

import numpy as np

def reshape_and_label_data(correct_file, wrong_file):
  """
  Reshapes data from .npy files into 3D arrays and assigns labels.

  Args:
    correct_file: Path to the .npy file containing correctly labeled data.
    wrong_file: Path to the .npy file containing incorrectly labeled data.

  Returns:
    A tuple containing:
      - X: A NumPy array with shape (samples, 30, 132) containing the reshaped data.
      - y: A NumPy array with shape (samples,) containing the labels (0 for incorrect, 1 for correct).
  """
  correct_data = np.load(correct_file)
  wrong_data = np.load(wrong_file)
  print(correct_data.shape)
  print(wrong_data.shape)
  # Reshape the data
  correct_data_reshaped = correct_data.reshape(-1, 30, 132)
  wrong_data_reshaped = wrong_data.reshape(-1, 30, 132)

  # Combine data and create labels
  X = np.concatenate((correct_data_reshaped, wrong_data_reshaped), axis=0)
  y = np.concatenate((np.ones(correct_data_reshaped.shape[0]), np.zeros(wrong_data_reshaped.shape[0])), axis=0)
  
  return X, y

# Example usage:
correct_file = r'C:\VS_Workspace\push-up-detection\output_data\processed_key\correct_keypoints\correct_keypoints_normalized_keypoints.npy'
wrong_file = r'C:\VS_Workspace\push-up-detection\output_data\processed_key\wrong_keypoints\wrong_keypoints_normalized_keypoints.npy' #replace with your wrong file path
X, y = reshape_and_label_data(correct_file, wrong_file)

print("X shape:", X.shape)
print("y shape:", y.shape)

(1500, 132)
(1500, 132)
X shape: (100, 30, 132)
y shape: (100,)


In [8]:
from sklearn.model_selection import train_test_split

# Assuming X and y are defined as in the previous code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 80% train, 20% test

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (80, 30, 132)
y_train shape: (80,)
X_test shape: (20, 30, 132)
y_test shape: (20,)


In [ ]:
# prompt: build cnn-lstm model with sequential 30 frames 

import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization


# Assuming X_train, X_test, y_train, y_test are defined as in the previous code

# Define the CNN-LSTM model
model = Sequential()

# CNN layers
model.add(Conv1D(filters=12, kernel_size=3, activation='relu', input_shape=(30, 132)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.25))  # Add dropout for regularization

model.add(Conv1D(filters=24, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.25))


# LSTM layer
model.add(LSTM(units=64))
model.add(Dropout(0.5))


# Dense layers
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
# model.summary()

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=12, validation_data=(X_test, y_test)) # Adjust epochs and batch_size



Epoch 1/10
7/7 [==============================] - 6s 154ms/step - loss: 0.6669 - accuracy: 0.6375 - val_loss: 0.7251 - val_accuracy: 0.4000
Epoch 2/10
7/7 [==============================] - 0s 16ms/step - loss: 0.5990 - accuracy: 0.7500 - val_loss: 0.7056 - val_accuracy: 0.4000
Epoch 3/10
7/7 [==============================] - 0s 15ms/step - loss: 0.5364 - accuracy: 0.8250 - val_loss: 0.7140 - val_accuracy: 0.4000
Epoch 4/10
7/7 [==============================] - 0s 11ms/step - loss: 0.4575 - accuracy: 0.8375 - val_loss: 0.6792 - val_accuracy: 0.6500
Epoch 5/10
7/7 [==============================] - 0s 12ms/step - loss: 0.4350 - accuracy: 0.8875 - val_loss: 0.6567 - val_accuracy: 0.7000
Epoch 6/10
7/7 [==============================] - 0s 14ms/step - loss: 0.3408 - accuracy: 0.8750 - val_loss: 0.6395 - val_accuracy: 0.6500
Epoch 7/10
7/7 [==============================] - 0s 14ms/step - loss: 0.3633 - accuracy: 0.8750 - val_loss: 0.6480 - val_accuracy: 0.6500
Epoch 8/10
7/7 [==========

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")